In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import json


In [20]:
driver = webdriver.Chrome()
driver.get("https://growithjane.com/explore-growlogs")

In [21]:
def scroll_page():
    """ Scroll the page and wait for items to load. """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(10)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height



In [22]:
scroll_page()  # Adjust time depending on the page's loading time

In [23]:
urls = set()
# Find the main container
main_container = driver.find_element(By.TAG_NAME, 'main')
# Find all the div elements with the class 'MuiGrid-spacing-xs-1'
div_elements = main_container.find_elements(By.CSS_SELECTOR, 'div.MuiGrid-spacing-xs-1 div')

for div in div_elements:
    try:
        # Find the <a> tag inside the div
        a_tag = div.find_element(By.TAG_NAME, 'a')
        urls.add(a_tag.get_attribute('href'))
    except Exception as e:
        continue


In [24]:
urls = list(urls)
print(f"Found {len(urls)} urls")

416

In [29]:
# Save URLs to a JSON file
with open('urls.json', 'w') as file:
    json.dump(urls, file, indent=4)

In [26]:
driver.quit()

In [20]:
# Load URLs from the JSON file
with open('urls.json', 'r') as file:
    urls = json.load(file)


In [49]:
def get_page_data(url, driver):
    driver.get(url)
    # Wait to load page
    time.sleep(2)

    try:
        main = driver.find_element(By.TAG_NAME, 'main')
        header_title = main.find_element(By.TAG_NAME, 'h1').text
        # Find the contents with the class 'environment-content'
        p_tags = driver.find_elements(By.CSS_SELECTOR, 'div.environment-content>p')
    except Exception as e:
        return None
    


    content_dict = {}
            
    for p_tag in p_tags:
        
        # Get the title from the <p> tag
        section = p_tag.text
        
        if section == 'TREE STAGES':
            continue

        # Get the next sibling <div> elements
        next_divs = []

        # Loop to find all following sibling <div> elements
        sibling = p_tag.find_element(By.XPATH, "following-sibling::*[1]")


        while sibling and sibling.tag_name != 'p':
            # Append the text of the <div> element to the list
            next_divs.append(sibling.text)
            try:
                # Move to the next sibling
                next = sibling.find_element(By.XPATH, "following-sibling::*[1]")
                sibling = next
                

            except:
                break
    
            
        content_dict[section] = next_divs

    content_dict['STRAIN'] = content_dict['STRAIN'][0]
    parts = content_dict['STRAIN'].split("\n")
    if len(parts) == 1:
        seed_bank = parts[0]
        seed_type = ''
    else:
        seed_bank, seed_type = parts

    content_dict['STRAIN'.lower()] = {'seed_bank': seed_bank, 'seed_type': seed_type}
    del content_dict['STRAIN']

    if 'MEDIUM' in content_dict:
        content_dict['MEDIUM'] = content_dict['MEDIUM'][0]
        parts = content_dict['MEDIUM'].split("\n")
        if len(parts) == 2:
            medium_title, medium_content = parts
            content_dict['MEDIUM'.lower()] = {'title': medium_title, 'content': medium_content}
        del content_dict['MEDIUM']

    environments = []
    for element in content_dict['ENVIRONMENT']:
        parts = element.split("\n")
        if len(parts) != 2:
            continue
        title, content = parts
        environments.append({"title": title, "content": content})
    content_dict['ENVIRONMENT'.lower()] = environments
    del content_dict['ENVIRONMENT']

    data = content_dict

    element = {"title": header_title, "data": data}

    return element

    

In [35]:
# Initialize the driver
driver = webdriver.Chrome()

In [44]:
results = []

In [50]:
print(f'number of urls: {len(urls)}')

for i in range(0, len(urls)):
    print(f"Processing {i}/{len(urls)}")
    data = get_page_data(urls[i], driver)
    results.append(data)

number of urls: 416
Processing 73/416
Processing 74/416
Processing 75/416
Processing 76/416
Processing 77/416
Processing 78/416
Processing 79/416
Processing 80/416
Processing 81/416
Processing 82/416
Processing 83/416
Processing 84/416
Processing 85/416
Processing 86/416
Processing 87/416
Processing 88/416
Processing 89/416
Processing 90/416
Processing 91/416
Processing 92/416
Processing 93/416
Processing 94/416
Processing 95/416
Processing 96/416
Processing 97/416
Processing 98/416
Processing 99/416
Processing 100/416
Processing 101/416
Processing 102/416
Processing 103/416
Processing 104/416
Processing 105/416
Processing 106/416
Processing 107/416
Processing 108/416
Processing 109/416
Processing 110/416
Processing 111/416
Processing 112/416
Processing 113/416
Processing 114/416
Processing 115/416
Processing 116/416
Processing 117/416
Processing 118/416
Processing 119/416
Processing 120/416
Processing 121/416
Processing 122/416
Processing 123/416
Processing 124/416
Processing 125/416


In [51]:
results = [item for item in results if item is not None]

In [52]:
# Save the results to a JSON file
with open('growlogs.json', 'w') as file:
    json.dump(results, file, indent=4)

In [31]:
# Cleanup
driver.quit()